In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


In [8]:
def unique_feat(xtr,xts,cols,target):
    t=xtr.copy()
    v=xts.copy()
    new_cols=[]
    for col in cols:
        
        d=t.groupby(col)[target].agg(['mean','count'])
        m=d['mean']
        c=d['count']
        med=np.median(c)
        d['new1']=m.apply(lambda x:1 if x==0 else 0)*c.apply(lambda x:1 if x>med else 0)
        d['new2']=m.apply(lambda x:2 if x==1 else 0)*c.apply(lambda x:1 if x>med else 0)
        d['new3']=m.apply(lambda x:3 if (x!=0) and (x!=1) else 0)*c.apply(lambda x:1 if x>med else 0)
        d['map_obj']=d['new1']+d['new2']+d['new3']+1
        t[col+'_UniqueTo']=t[col].map(d['map_obj'])
        v[col+'_UniqueTo']=v[col].map(d['map_obj'])
        v[col+'_UniqueTo']=v[col+'_UniqueTo'].fillna(0)
        new_cols.append(col+'_UniqueTo')
    return t,v,new_cols

In [9]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [10]:
def AST_ecoding(train,test,columns,target,weight):
    t=train.copy()
    v=test.copy()
    gm=t[target].mean()
    for col in columns:
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        t[col+'_enc']=t[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    t=t.drop(columns,axis=1)
    return t,v

In [11]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
lgb=LGBMClassifier(n_estimators=200)
lr=LogisticRegression()

In [12]:
train,test=train_test_split(df1,test_size=0.3,random_state=123)

columns=list(df1.columns)
columns.remove('target')
target='target'
t,v,new_cols=unique_feat(train,test,columns,target)

In [13]:
t.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12,RESOURCE_UniqueTo,MGR_ID_UniqueTo,ROLE_DEPTNAME_UniqueTo,ROLE_FAMILY_DESC_UniqueTo,ROLE_CODE_UniqueTo,ROLLUP_12_UniqueTo
7693,25285,51761,120370,117906,117908,0,117961118413,2,4,4,4,4,4
17676,79168,25396,117941,117913,117888,0,118602118603,4,2,4,4,4,4
1969,33329,8028,124948,130134,120954,0,117902117903,2,2,4,4,2,4
15050,3690,3692,126229,207709,121596,0,117961118413,1,2,4,2,4,4
345,77203,15621,120943,126684,126685,1,117961119256,4,1,4,4,4,4


In [14]:
org_feat=['RESOURCE','MGR_ID','ROLE_DEPTNAME','ROLE_FAMILY_DESC','ROLE_CODE','ROLLUP_12']
t,v=AST_ecoding(t,v,org_feat,target,20)
t.head()

,target,RESOURCE_UniqueTo,MGR_ID_UniqueTo,ROLE_DEPTNAME_UniqueTo,ROLE_FAMILY_DESC_UniqueTo,ROLE_CODE_UniqueTo,ROLLUP_12_UniqueTo,RESOURCE_enc,MGR_ID_enc,ROLE_DEPTNAME_enc,ROLE_FAMILY_DESC_enc,ROLE_CODE_enc,ROLLUP_12_enc
7693,0,2,4,4,4,4,4,0.052553,0.156247,0.038472,0.065241,0.035277,0.028338
17676,0,4,2,4,4,4,4,0.129880,0.039868,0.076784,0.033729,0.036288,0.120699
1969,0,2,2,4,4,2,4,0.038539,0.046246,0.101101,0.038227,0.010142,0.036647
15050,0,1,2,4,2,4,4,0.055055,0.023123,0.033690,0.037295,0.130292,0.028338
345,1,4,1,4,4,4,4,0.092828,0.093746,0.070443,0.148434,0.182512,0.108729


In [15]:
new_cols

['RESOURCE_UniqueTo',
 'MGR_ID_UniqueTo',
 'ROLE_DEPTNAME_UniqueTo',
 'ROLE_FAMILY_DESC_UniqueTo',
 'ROLE_CODE_UniqueTo',
 'ROLLUP_12_UniqueTo']

In [16]:
t['t?']=1
v['t?']=0
d=pd.concat([t,v])
d=pd.get_dummies(d,columns=new_cols)
t=d[d['t?']==1].drop('t?',axis=1).copy()
v=d[d['t?']==0].drop('t?',axis=1).copy()
del d

In [17]:

xtr=t.drop('target',axis=1)
xts=v.drop(target,axis=1)
ytr=t[target]
yts=v[target]
cols=list(xtr.columns)


In [28]:
def eliminate(algo,cols,xtr,ytr,xts,yts):
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    best_score=auc
    high=0
    removed=[]
    for i in tqdm(cols):
        c=cols.copy()
        c.remove(i)
        if len(c)>1:
            mod=algo.fit(xtr[c],ytr)
            auc_sc=aucroc(mod,xts[c],yts)
            if auc_sc>=best_score:
                removed.append(i)
            if auc_sc>high:
                high=auc_sc
                r=i
    return high,i

In [19]:
xts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9831 entries, 5666 to 12899
Data columns (total 32 columns):
RESOURCE_enc                     9831 non-null float64
MGR_ID_enc                       9831 non-null float64
ROLE_DEPTNAME_enc                9831 non-null float64
ROLE_FAMILY_DESC_enc             9831 non-null float64
ROLE_CODE_enc                    9831 non-null float64
ROLLUP_12_enc                    9831 non-null float64
RESOURCE_UniqueTo_0.0            9831 non-null uint8
RESOURCE_UniqueTo_1.0            9831 non-null uint8
RESOURCE_UniqueTo_2.0            9831 non-null uint8
RESOURCE_UniqueTo_3.0            9831 non-null uint8
RESOURCE_UniqueTo_4.0            9831 non-null uint8
MGR_ID_UniqueTo_0.0              9831 non-null uint8
MGR_ID_UniqueTo_1.0              9831 non-null uint8
MGR_ID_UniqueTo_2.0              9831 non-null uint8
MGR_ID_UniqueTo_3.0              9831 non-null uint8
MGR_ID_UniqueTo_4.0              9831 non-null uint8
ROLE_DEPTNAME_UniqueTo_0.0   

In [20]:
algo=LogisticRegression(max_iter=10e7)
removed=[]
cols=list(xtr.columns)
mod=algo.fit(xtr[cols],ytr)
auc=aucroc(mod,xts[cols],yts)
high_current=auc
high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
while high_new>high_current:
    cols.remove(r)
    removed.append(r)
    high_current=high_new
    high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
    print(high_new)
print(high_current)
print(removed)

100%|██████████| 31/31 [00:10<00:00,  2.89it/s]


0.7950017588785543
0.7950023262587329
['ROLLUP_12_UniqueTo_4.0']


In [29]:
def col_select(algo,step):
    
    
    cols=list(xtr.columns)
    p=[]
    removed=[]
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    high_current=auc
    removed_new=[]
    high_new=auc
    while high_current-high_new<step:    


        removed=removed+removed_new
        print(len(removed))
        removed_new=[]


        if high_current==auc:
            pass
        else:
            high_current=high_new

        high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
        while high_new>=high_current:
            cols.remove(r)
            removed_new.append(r)
            high_current=high_new
            high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
            print(high_new)
        
    print('high_auc=',high_current)
    print('auc_one step ahead=',high_new)
    print('removed: ',removed)
    print('len=',len(removed))

In [30]:
lr=LogisticRegression(max_iter=10e7)
col_select(lr,0.05)

0


100%|██████████| 31/31 [00:12<00:00,  2.49it/s]


0.7950017588785543
1


100%|██████████| 30/30 [00:12<00:00,  2.36it/s]


0.7948245471360541
2


...

0.8507211402072072
28


100%|██████████| 3/3 [00:00<00:00, 17.49it/s]


0.8359195152303752
29


100%|██████████| 2/2 [00:00<?, ?it/s]


0
high_auc= 0.8359195152303752
auc_one step ahead= 0
removed:  ['ROLLUP_12_UniqueTo_4.0', 'ROLLUP_12_UniqueTo_2.0', 'ROLLUP_12_UniqueTo_1.0', 'ROLLUP_12_UniqueTo_0.0', 'ROLE_CODE_UniqueTo_4.0', 'ROLE_CODE_UniqueTo_2.0', 'ROLE_CODE_UniqueTo_1.0', 'ROLE_CODE_UniqueTo_0.0', 'ROLE_FAMILY_DESC_UniqueTo_4.0', 'ROLE_FAMILY_DESC_UniqueTo_2.0', 'ROLE_FAMILY_DESC_UniqueTo_1.0', 'ROLE_FAMILY_DESC_UniqueTo_0.0', 'ROLE_DEPTNAME_UniqueTo_4.0', 'ROLE_DEPTNAME_UniqueTo_2.0', 'ROLE_DEPTNAME_UniqueTo_1.0', 'ROLE_DEPTNAME_UniqueTo_0.0', 'MGR_ID_UniqueTo_4.0', 'MGR_ID_UniqueTo_3.0', 'MGR_ID_UniqueTo_2.0', 'MGR_ID_UniqueTo_1.0', 'MGR_ID_UniqueTo_0.0', 'RESOURCE_UniqueTo_4.0', 'RESOURCE_UniqueTo_3.0', 'RESOURCE_UniqueTo_2.0', 'RESOURCE_UniqueTo_1.0', 'RESOURCE_UniqueTo_0.0', 'ROLLUP_12_enc', 'ROLE_CODE_enc', 'ROLE_FAMILY_DESC_enc']
len= 29


In [ ]:
col_select(lgb,0.005)

In [27]:
removed

['ROLLUP_12_UniqueTo_4.0',
 'ROLLUP_12_UniqueTo_2.0',
 'ROLLUP_12_UniqueTo_1.0',
 'ROLLUP_12_UniqueTo_0.0',
 'ROLE_CODE_UniqueTo_4.0']